# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/statics/reclassified_municipality_mapping.ipynb"
    ]
}
```

# **TABLES**

---

### STAGES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE OBJECT_ID = OBJECT_ID(N'[DBO].[SEQ_VWSSTAGE_CBS_POPULATION_BASE]') AND TYPE = 'SO')
CREATE SEQUENCE SEQ_VWSSTAGE_CBS_POPULATION_BASE
  START WITH 1
  INCREMENT BY 1;
GO

-- 2) CREATE STAGE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[CBS_POPULATION_BASE]'))
CREATE TABLE [VWSSTAGE].[CBS_POPULATION_BASE](
	[ID] INT PRIMARY KEY NONCLUSTERED NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTAGE_CBS_POPULATION_BASE]),
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE(),
  [GEMEENTE_CODE]  VARCHAR(100) NULL,
  [GEMEENTE] VARCHAR(100) NULL,
  [LEEFTIJD] VARCHAR(100) NULL,
  [GESLACHT] VARCHAR(100) NULL,
  [DATUM_PEILING] VARCHAR(100) NULL,
  [POPULATIE] VARCHAR(100) NULL,
  [VEILIGHEIDSREGIO_CODE] VARCHAR(100) NULL,
  [VEILIGHEIDSREGIO_NAAM] VARCHAR(100) NULL,
  [PROVINCIE_CODE] VARCHAR(100) NULL,
  [PROVINCIE_NAAM] VARCHAR(100) NULL,
  [GGD_CODE] VARCHAR(100) NULL,
  [GGD_NAAM] VARCHAR(100) NULL
);
GO

-- 3) CREATE STAGE INDEXE(S).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCIX_DLI_VWSSTAGE_CBS_POPULATION_BASE')
CREATE NONCLUSTERED INDEX [NCIX_DLI_VWSSTAGE_CBS_POPULATION_BASE] ON [VWSSTAGE].[CBS_POPULATION_BASE] (
  [DATE_LAST_INSERTED]
);
GO

IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCI_DLI_VWSSTAGE_CIMS_VACCINATED_AGE_GROUP')
CREATE NONCLUSTERED INDEX [NCI_DLI_VWSSTAGE_CIMS_VACCINATED_AGE_GROUP] ON [VWSSTAGE].[CBS_POPULATION_BASE] (
  [DATE_LAST_INSERTED], 
  [GEMEENTE_CODE], 
  [GEMEENTE], 
  [LEEFTIJD], 
  [GESLACHT], 
  [DATUM_PEILING], 
  [POPULATIE], 
  [VEILIGHEIDSREGIO_CODE], 
  [VEILIGHEIDSREGIO_NAAM], 
  [PROVINCIE_CODE], 
  [PROVINCIE_NAAM], 
  [GGD_CODE], 
  [GGD_NAAM]
) WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY];
GO

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSSTATIC_CBS_POPULATION_BASE]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSSTATIC_CBS_POPULATION_BASE]
	START WITH 1
	INCREMENT BY 1;
GO

-- 2) CREATE STAGE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[CBS_POPULATION_BASE]'))
CREATE TABLE [VWSSTATIC].[CBS_POPULATION_BASE](
	[ID] [INT] PRIMARY KEY NONCLUSTERED NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTATIC_CBS_POPULATION_BASE]),
	[GEMEENTE_CODE]  [VARCHAR](10) NULL,
	[GEMEENTE] [VARCHAR](100) NULL,
	[LEEFTIJD] [INT] NULL,
	[GESLACHT] [VARCHAR](10) NULL,
	[DATUM_PEILING] [DATE] NULL,
	[POPULATIE] [INT] NULL,
	[VEILIGHEIDSREGIO_CODE] [VARCHAR](10) NULL,
	[VEILIGHEIDSREGIO_NAAM] [VARCHAR](100) NULL,
	[PROVINCIE_CODE] [VARCHAR](10) NULL,
	[PROVINCIE_NAAM] [VARCHAR](100) NULL,
	[GGD_CODE] [VARCHAR](10) NULL,
	[GGD_NAAM] [VARCHAR](100) NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

-- 3) CREATE STAGE INDEXE(S).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCIX_DLI_VWSSTATIC_CBS_POPULATION_BASE')
CREATE NONCLUSTERED INDEX [NCIX_DLI_VWSSTATIC_CBS_POPULATION_BASE] ON [VWSSTATIC].[CBS_POPULATION_BASE] (
    [DATE_LAST_INSERTED] ASC
);
GO


# **STORED PROCEDURES**

---

In 2022 multiple municipalties (i.e. gemeentes) are reclassified (e.g. merging) into novel muncipalities. To implement these changes, several database adjustments had to be performed.

Within the `[DBO].[SP_CBS_POPULATION_BASE_STATIC]`, with help of the mapping-tabel (i.e. `[RECLASSIFIED_MUNICIPALITY_MAPPING]` ), the reclassified muncipalities will be progated towards the `[VWSSTATIC].[CBS_POPULATION_BASE]`:

```sql
LEFT JOIN (
    SELECT 
        * 
    FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
    WHERE [ACTIVE] = 1 
        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
) B ON A.[GEMEENTE_CODE]=B.[GM_CODE_ORIGINAL]
```

In turn, all downstream procedures, views or tabel - depending on `[VWSSTATIC].[CBS_POPULATION_BASE]` - will automatcally implement the modified muncipality codes (i.e. `GM_CODE`, `GMCODE` or `GEMEENTE_CODE`).

### STAGING &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_CBS_POPULATION_BASE_STATIC]
AS
BEGIN

-- POPULATION_BASE IS TRANSFERRED FROM STAGE TO STATIC, THE TABLE RECODE_MUNICIPALITY IS USED TO APPLY CHANGES IN THE MUNICIPALITIES.
-- THE RECODE_MUNICIPALITY TABLE IS LEFT-JOINED, SO ONLY MUNICIPALITIES THAT MET WITH A CHANGE WILL BE AFFECTED. 
-- ONLY CHANGES THAT ARE 'ACTIVE' ARE SELECTED.
INSERT INTO [VWSSTATIC].[CBS_POPULATION_BASE] (
    [GEMEENTE_CODE]
    ,[GEMEENTE]
    ,[LEEFTIJD]
    ,[GESLACHT]
    ,[DATUM_PEILING]
    ,[POPULATIE]
    ,[VEILIGHEIDSREGIO_CODE]
    ,[VEILIGHEIDSREGIO_NAAM]
    ,[PROVINCIE_CODE]
    ,[PROVINCIE_NAAM]
    ,[GGD_CODE]
    ,[GGD_NAAM]
)
SELECT 
    ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE]) AS [GEMEENTE_CODE]
    ,ISNULL(B.[GM_NAME_NEW],[GEMEENTE]) AS [GEMEENTE]
    ,A.[LEEFTIJD]
    ,A.[GESLACHT]
    ,CAST(A.[DATUM_PEILING] AS DATE) AS [DATUM_PEILING]
    ,ROUND(SUM(CAST(A.[POPULATIE] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS [POPULATIE] 
    ,ISNULL(B.[VR_CODE],[VEILIGHEIDSREGIO_CODE]) AS [VEILIGHEIDSREGIO_CODE]
    ,ISNULL(B.[VR_NAME],[VEILIGHEIDSREGIO_NAAM]) AS [VEILIGHEIDSREGIO_NAAM]
    ,ISNULL(B.[PROVINCE_CODE],[PROVINCIE_CODE]) AS [PROVINCIE_CODE]
    ,ISNULL(B.[PROVINCE_NAME],[PROVINCIE_NAAM]) AS [PROVINCIE_NAAM]
    ,ISNULL(B.[GGD_CODE], A.[GGD_CODE]) AS [GGD_CODE]
    ,ISNULL(B.[GGD_NAME],[GGD_NAAM]) AS [GGD_NAAM]
FROM [VWSSTAGE].[CBS_POPULATION_BASE] A
    LEFT JOIN (
        SELECT 
            * 
        FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
        WHERE [ACTIVE] = 1 
            AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
    ) B ON A.[GEMEENTE_CODE] = B.[GM_CODE_ORIGINAL]
WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CBS_POPULATION_BASE])
GROUP BY
    ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE]) 
    ,ISNULL(B.[GM_NAME_NEW],[GEMEENTE]) 
    ,A.[LEEFTIJD]
    ,A.[GESLACHT]
    ,CAST(A.[DATUM_PEILING] AS DATE) 
    ,ISNULL(B.[VR_CODE],[VEILIGHEIDSREGIO_CODE]) 
    ,ISNULL(B.[VR_NAME],[VEILIGHEIDSREGIO_NAAM]) 
    ,ISNULL(B.[PROVINCE_CODE],[PROVINCIE_CODE]) 
    ,ISNULL(B.[PROVINCE_NAME],[PROVINCIE_NAAM]) 
    ,ISNULL(B.[GGD_CODE],A.[GGD_CODE]) 
    ,ISNULL(B.[GGD_NAME],[GGD_NAAM]) 
ORDER BY [DATUM_PEILING], ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE])
END;
GO

# **DATATINO CONFIGURATIONS**

---


### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'STATIC_DATA',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

-- 2) UPSERT WORKFLOW(S).....
SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @SCHEDULE = '0 12 * */3 MON-FRI', -- AT 12:00 PM, EVERY 3 MONTHS
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/STATIC/populatie_data.csv'
    WHEN 'acceptance' THEN 'datafiles/STATIC/populatie_data.csv'
    ELSE 'datafiles/STATIC/populatie_data.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'AzureBlob'
    WHEN 'acceptance' THEN 'AzureBlob'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = 'SOURCE_Load_Population_Base';
SET @source_description = CONCAT('LOAD: CBS_POPULATION_BASE - ', @workflow_name);
SET @target_name = 'VWSSTAGE.CBS_POPULATION_BASE';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'Gemeente_code|Gemeente|Leeftijd|Geslacht|Datum_peiling|Populatie|Veiligheidsregio_code|Veiligheidsregio_naam|Provincie_code|Provincie_naam|GGD_code|GGD_naam',
    @target_columns = 'GEMEENTE_CODE|GEMEENTE|LEEFTIJD|GESLACHT|DATUM_PEILING|POPULATIE|VEILIGHEIDSREGIO_CODE|VEILIGHEIDSREGIO_NAAM|PROVINCIE_CODE|PROVINCIE_NAAM|GGD_CODE|GGD_NAAM',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'Colon',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO STATIC......
SET @source = 'DBO.SP_CBS_POPULATION_BASE_STATIC';
SET @source_name = 'SOURCE_STATIC_POPULATION_BASE';
SET @source_description = 'MAP: CBS_POPULATION_BASE FROM STAGE TO STATIC';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = 'PROCESS_LOAD_POPULATION_BASE';
SET @process_description = CONCAT('LOAD: CBS_POPULATION_BASE - ', @workflow_name);
SET @process_source_name = 'SOURCE_Load_Population_Base';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO STATIC......
SET @process_name = 'PROCESS_STATIC_POPULATION_BASE';
SET @process_description = 'MAP: CBS_POPULATION_BASE FROM STAGE TO STATIC';
SET @process_source_name = 'SOURCE_STATIC_POPULATION_BASE';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = 'DEPENDENCY_PROCESS_STATIC_POPULATION_BASE';
SET @dependency_dataflow_name = 'PROCESS_STATIC_POPULATION_BASE';
SET @dependency_process_name = 'PROCESS_LOAD_POPULATION_BASE';
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO